# ANALISI E REVISIONE DEL PROGETTO LARSPLITTING 2D 

## CLASSE REFACTORING: spaceIndex

Variabili utili per testare il funzionamento


# FUNZIONI AGGIUNTIVE

### RemoveIntersection

rimuove le intersezioni contenute in **covers** che i boundingbox hanno con se stessi



In [8]:
function removeIntersection(covers::Array{Array{Int64,1},1})
    @threads for k=1:length(covers)
        covers[k] = setdiff(covers[k],[k])	#toglie le intersezioni con se stesso 
    end
end

removeIntersection (generic function with 1 method)

In [17]:
@btime removeIntersection(cov)    # 7.011 μs

  7.255 μs (51 allocations: 3.44 KiB)


### CreateIntervalTree

funzione che crea un albero di supporto per la funzione principale **boxcovering**; nel particolare dato un dizionario ordinato crea un intervalTrees cioè una struttura dati che contiene intervalli e che consente di trovare in modo efficiente tutti gli intervalli che si sovrappongono a un determinato intervallo o punto.

In [9]:
function createIntervalTree(boxdict::AbstractDict{Array{Float64,1},Array{Int64,1}})
    tree = IntervalTrees.IntervalMap{Float64,Array}()
    for (key, boxset) in boxdict
        tree[tuple(key...)] = boxset
    end
    return tree
end

createIntervalTree (generic function with 1 method)

## addIntersection

addIntersection aggiunge in 'covers' in i-esima posizione tutti i bounding box che intersecano l'i-esimo bounding box


## TEST

In [ ]:
@testset "Refactoring spaceindex tests" begin
    V,(VV,EV,FV) = Lar.cuboidGrid([2,1],true)
    EV = [[1, 2], [3, 4], [5, 6], [1, 3], [2, 4], [3, 5], [4, 6]]
    cellpoints = [ V[:,EV[k]]::Lar.Points for k=1:length(EV) ]
    bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
    xboxdict = Lar.coordintervals(1,bboxes)
    yboxdict = Lar.coordintervals(2,bboxes)
    xs = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in xboxdict
        xs[tuple(key...)] = boxset
    end
    ys = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in yboxdict
        ys[tuple(key...)] = boxset
    end
    xcovers = Lar.boxcovering(bboxes, 1, xs)
    ycovers = Lar.boxcovering(bboxes, 2, ys)
    covers = [intersect(pair...) for pair in zip(xcovers,ycovers)]
    
    @test covers == Array{Int64,1}[[1, 4, 5], [4, 5, 2, 6, 7], [6, 7, 3], 
        [1, 4, 2, 6], [1, 5, 2, 7], [4, 2, 6, 3], [5, 2, 7, 3]]
end

![test di spaceindex](https://github.com/MarcoCap13/LAR-SPLITTING-2D-5.b-/blob/main/docs/test/spaceindex_test.png?raw=true)